## Decorators
- By definition, a decorator is a function that takes another function and extends the behavior of the latter function without explicitly modifying it.

### First Class Objects
- In Python, functions are first-class objects. This means that functions can be passed around and used as arguments, just like any other object (string, int, float, list, and so on).

In [1]:
def say_hello(name):
    return f"Hello {name}"

def be_awesome(name):
    return f"Yo {name}, together we are the awesomest!"

def greet_bob(greeter_func):
    return greeter_func("Bob")

Here, `say_hello()` and `be_awesome()` are regular functions that expect a name given as a string. The `greet_bob()` function however, expects a function as its argument. We can, for instance, pass it the `say_hello()` or the `be_awesome()` function:

In [2]:
print(greet_bob(say_hello))
print(greet_bob(be_awesome))

Hello Bob
Yo Bob, together we are the awesomest!


Note that `greet_bob(say_hello)` refers to two functions, but in different ways: `greet_bob()` and `say_hello`. The `say_hello` function is named without parentheses. This means that only a reference to the function is passed. The function is not executed. The `greet_bob()` function, on the other hand, is written with parentheses, so it will be called as usual.

### Inner Functions
- It’s possible to define functions inside other functions. Such functions are called inner functions.

In [3]:
def parent():
    print("Printing from the parent() function")

    def first_child():
        print("Printing from the first_child() function")

    def second_child():
        print("Printing from the second_child() function")

    second_child()
    first_child()

parent()

Printing from the parent() function
Printing from the second_child() function
Printing from the first_child() function


The inner functions are not defined until the parent function is called. They are locally scoped to parent(): they only exist inside the parent() function as local variables.

### Returning Functions From Functions
- Python also allows you to use functions as return values. 

In [4]:
def parent(num):
    def first_child():
        return "Hi, I am Emma"

    def second_child():
        return "Call me Liam"

    if num == 1:
        return first_child
    else:
        return second_child

first = parent(1)
second = parent(2)
print(first)
print(second)
print(first())
print(second())

<function parent.<locals>.first_child at 0x06103B68>
<function parent.<locals>.second_child at 0x01145100>
Hi, I am Emma
Call me Liam


We are returning `first_child` without the parentheses. Recall that this means that you are returning a reference to the function `first_child`. In contrast `first_child()` with parentheses refers to the result of evaluating the function.

The somewhat cryptic output simply means that the `first` variable refers to the local `first_child()` function inside of `parent()`, while `second` points to `second_child()`.

Finally, note that in the earlier example you executed the inner functions within the `parent` function, for instance `first_child()`. However, in this last example, you did not add parentheses to the inner functions—`first_child`—upon returning. That way, you got a reference to each function that you could call in the future.

## Simple Decorators

In [5]:
def my_decorator(func):
    def wrapper():
        print("Something is happening before the function is called.")
        func()
        print("Something is happening after the function is called.")
    return wrapper

def say_whee():
    print("Whee!")

say_whee = my_decorator(say_whee)

print(say_whee())
print(say_whee)

Something is happening before the function is called.
Whee!
Something is happening after the function is called.
None
<function my_decorator.<locals>.wrapper at 0x01145190>


- The so-called decoration happens using: `say_whee = my_decorator(say_whee)`
- In effect, the name `say_whee` now points to the `wrapper()` inner function.
- However, `wrapper()` has a reference to the original `say_whee()` as `func`, and calls that function between the two calls to print().
- `wrapper()` is a decorator function which took `say_whee()` function and extended the behaviour of `say_whee()`
- `say_whee()` earlier only printed "Whee!"
- After being decorated by `wrapper()` function, call to `say_whee()` now add a print statement before and after printing "Whee!"
- ### Put simply: decorators wrap a function, modifying its behavior.

In [6]:
from datetime import datetime

def not_during_the_night(func):
    def wrapper():
        if 7 <= datetime.now().hour < 22:
            func()
        else:
            pass  # Hush, the neighbors are asleep
    return wrapper

def say_whee():
    print("Whee!")

say_whee = not_during_the_night(say_whee)

say_whee()

Whee!


- Because `wrapper()` is a regular Python function, the way a decorator modifies a function can change dynamically. So as not to disturb your neighbors, the following example will only run the decorated code during the day.
- If you try to call `say_whee()` after bedtime, nothing will happen.

In [7]:
def my_decorator(func):
    def wrapper():
        print("Something is happening before the function is called.")
        func()
        print("Something is happening after the function is called.")
    return wrapper

@my_decorator
def say_whee():
    print("Whee!")

say_whee()

Something is happening before the function is called.
Whee!
Something is happening after the function is called.


- Python allows you to use decorators in a simpler way with the `@` symbol, sometimes called the “pie” syntax.
- So, `@my_decorator` is just an easier way of saying say_whee = `my_decorator(say_whee)`. It’s how you apply a decorator to a function.

### Reusing Decorators
- Let’s move the decorator to its own module that can be used in many other functions.

In [8]:
def do_twice(func):
    def wrapper_do_twice():
        func()
        func()
    return wrapper_do_twice

# If we keep our decorator in its own module naming decorators, then we can use it using
# from decorators import do_twice

@do_twice
def say_whee():
    print("Whee!")

say_whee()

Whee!
Whee!


# General decorator pattern

In [9]:
import functools

def decorator(func):
    @functools.wraps(func) # It will preserve the information about original function
    def wrapper_decorator(*args, **kwargs): # So that inner can take any number of arguments
        # Do something before
        value = func(*args, **kwargs)
        # Do something after
        return value # So that inner argument do not eat return value of original function
    return wrapper_decorator

In [10]:
import functools
import time

def timer(func):
    @functools.wraps(func)
    def wrapper_timer(*args, **kwargs):
        start = time.perf_counter()
        value = func(*args, **kwargs)
        end = time.perf_counter()
        run_time = end - start
        print(f"Took {run_time:.4f} to run {func.__name__}")
        return value
    return wrapper_timer

@timer
def waste_some_time(c):
    for _ in range(c):
        # If count is 1, sum will be calculated only 1 time.
        # If count is 999, sum will be calculated only 999 time.
        sum([i**2 for i in range(999)]) 

waste_some_time(1)

Took 0.0006 to run waste_some_time


In [11]:
waste_some_time(99)

Took 0.0410 to run waste_some_time
